In [227]:
import iris

import numpy as np
import scipy.stats

In [228]:
%load_ext autoreload
%autoreload 2
import math_helpers

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [229]:
era5 = iris.load("testing_data/ERA5.nc")
obs = era5[1]

cm_hist = iris.load_cube("testing_data/CMIP6/medium_area/historical_monthly_surface_temp_single_levels_IPSL-CM5A2-INCA/tas_Amon_IPSL-CM5A2-INCA_historical_r1i1p1f1_gr_19790116-20141216_v20200729.nc")
cm_future = iris.load_cube("testing_data/CMIP6/medium_area/SSP3-7_monthly_surface_temp_single_levels_IPSL-CM5A2-INCA/tas_Amon_IPSL-CM5A2-INCA_ssp370_r1i1p1f1_gr_20150116-20501216_v20201218.nc")

/home/jakobwes/anaconda3/lib/python3.8/site-packages/iris/fileformats/cf.py:859: UserWarning: Missing CF-netCDF measure variable 'areacella', referenced by netCDF variable 'tas'
  warnings.warn(
/home/jakobwes/anaconda3/lib/python3.8/site-packages/iris/fileformats/cf.py:859: UserWarning: Missing CF-netCDF measure variable 'areacella', referenced by netCDF variable 'tas'
  warnings.warn(


In [230]:
# Regrid
cm_hist = cm_hist.regrid(obs, iris.analysis.Linear())
cm_future = cm_future.regrid(obs, iris.analysis.Linear())

In [231]:
# Get data
obs = np.array(obs.data[:, 0, :, :])
cm_hist = cm_hist.data
cm_future = cm_future.data

Test ISIMIP:

In [232]:
obs = obs[:, 1, 1]
cm_hist = cm_hist[:, 1, 1]
cm_future = cm_future[:, 1, 1]

In [235]:
from isimip import ISIMIP

temp_debiaser = ISIMIP(variable = "tas")
temp_debiaser.trend_preservation = "mixed"
temp_debiaser.lower_bound = 0
#temp_debiaser.upper_bound = 300
temp_debiaser.lower_threshold = 50
#temp_debiaser.upper_treshold = 290

In [236]:
obs_future = temp_debiaser.step5(obs, cm_hist, cm_future)
test = temp_debiaser.step6(obs, obs_future, cm_hist, cm_future)

In [238]:
test

masked_array(data=[279.170654296875, 278.0284423828125, 281.4710388183594,
                   281.5895080566406, 287.8841552734375, 295.250244140625,
                   296.6181640625, 297.65765380859375, 290.98394775390625,
                   286.3597106933594, 284.252685546875, 279.1689147949219,
                   278.6732482910156, 278.9498596191406,
                   280.7474365234375, 283.1839294433594,
                   286.0945739746094, 296.8501892089844,
                   299.40203857421875, 298.65631103515625, 294.783203125,
                   288.32568359375, 282.20697021484375, 280.66064453125,
                   0.0, 279.9809265136719, 283.1078186035156,
                   282.01885986328125, 285.25689697265625,
                   293.2889099121094, 299.85101318359375,
                   300.85076904296875, 292.18603515625, 285.3833923339844,
                   281.9659729003906, 279.63629150390625,
                   279.1618347167969, 279.8487548828125,
             

In [193]:
test

masked_array(data=[0.0, 0.0, 274.7698974609375, 274.8809509277344,
                   275.245849609375, 276.7658386230469, 276.9475402832031,
                   276.9544982910156, 277.04901123046875, 277.1767578125,
                   277.4232177734375, 277.5163879394531,
                   277.71160888671875, 277.7563781738281,
                   277.849853515625, 277.89813232421875,
                   277.9290771484375, 278.0284423828125,
                   278.1370849609375, 278.1484375, 278.1800537109375,
                   278.20526123046875, 278.294921875, 278.33831787109375,
                   278.4313659667969, 278.6287536621094,
                   278.6636047363281, 278.6700439453125,
                   278.6732482910156, 278.6995849609375,
                   278.7330017089844, 278.780517578125, 278.8120422363281,
                   278.8861083984375, 278.8876953125, 278.91204833984375,
                   278.9498596191406, 279.0223388671875,
                   279.03054809570

In [153]:
test

masked_array(data=[0.0, 0.0, nan, nan, nan, nan, nan, nan, nan, nan, nan,
                   nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
                   nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
                   nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
                   nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
                   nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
                   nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
                   nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
                   nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
                   nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
                   nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
                   nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
                   nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
                   nan, nan, nan, nan,

In [11]:
from switanek2017 import Switanek2017
temp_debiaser = Switanek2017()
debiased_cm5 = temp_debiaser.absolute_sdm_location(obs[:,1,1], cm_hist[:,1,1], cm_future[:,1,1])

In [12]:
debiased_cm4-debiased_cm5

masked_array(data=[561.1606146967908, 620.9720802263178,
                   528.4494059040615, 507.51448391259953,
                   286.24350013101594, 226.47546302322257,
                   519.216175969339, 504.85053038784656,
                   398.08813378712887, 388.9076985268727,
                   611.6611879105994, 574.8041831352879,
                   731.9400575651226, 610.8525026704112, 532.451663773567,
                   532.4325396570428, 445.82052673763366,
                   466.5710558926103, 135.52900221796563,
                   228.95845162649402, 172.28739122473922,
                   316.81877140666643, 566.539154966927,
                   514.4426552476037, 1527.2428933125457,
                   458.0565998563618, 526.7478147240492,
                   547.5320652215295, 586.907999718904,
                   207.70759814968906, 24.10298253875419,
                   -15.561038509597893, 262.719525957899,
                   566.4768578964043, 520.4981178623873, 448